
Problem Statement:
Build a chatbot that answers general health-related questions using a Large Language Model while ensuring responses are safe, friendly, and non-diagnostic.

Goal:
Use prompt engineering and safety rules to provide reliable general health information without giving harmful medical advice.




In [1]:
#Install Required Libraries
!pip install transformers accelerate torch

In [2]:
#Load Mistral-7B-Instruct Model
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [11]:
#Safety Filter
def safety_filter(query):
    blocked_terms = [
        "dosage", "dose", "prescription", "diagnose",
        "treatment", "cure", "medicine", "medication", "drug"
    ]
    return not any(term in query.lower() for term in blocked_terms)


In [4]:
#Prompt Engineering
def build_prompt(user_query):
    return f"""
[INST]
You are a friendly and helpful medical assistant.
Provide general health information only.
Do NOT diagnose diseases or suggest medications.
Encourage consulting a healthcare professional when appropriate.

User question: {user_query}
[/INST]
"""

In [13]:
#ChatBot Function
def health_chatbot(query):
    if not safety_filter(query):
        return f"{query}\n\nI can share general health information, but for diagnosis or treatment, please consult a healthcare professional."

    prompt = build_prompt(query)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=True,
        temperature=0.5,
        eos_token_id=tokenizer.encode("[/INST]")[0]
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = response.replace("[INST]", "").replace("[/INST]", "").strip()
    response = response.split("User question:")[-1].strip()
    return f"{query}\n\n{response}"


In [14]:
#Example queries
print(health_chatbot("What causes a sore throat?"))
print(health_chatbot("Is paracetamol safe for children?"))
print(health_chatbot("What medication should I use for my child's fever?"))

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


What causes a sore throat?

What causes a sore throat?

A sore throat can be caused by a variety of factors, including viral infections such as the common cold or flu, bacterial infections, tonsillitis, strep throat, mononucleosis, and even allergies. It's important to note that a sore throat can also be a symptom of more serious conditions, so it's always a good idea to consult with a healthcare professional if you're experiencing a sore throat that persists or is severe.
Is paracetamol safe for children?

Is paracetamol safe for children?

Paracetamol is generally safe for children when used appropriately. It is commonly used to relieve pain and reduce fever. However, it's important to follow the recommended dosage and always consult with a healthcare professional before giving any medication to a child, especially if they have a pre-existing medical condition or are taking other medications. The dosage for children varies depending on their age and weight, so it's important to check

Explanation of Results & Final Insights

The General Health Query Chatbot demonstrates how a large language model (Mistral-7B-Instruct) can provide clear and friendly general health information. Prompt engineering ensures the chatbot answers user queries in a professional, helpful tone while guiding users to consult healthcare professionals when necessary. Safety filters effectively block questions requesting specific medications, dosages, or treatments, preventing potentially harmful advice.

Example queries like “What causes a sore throat?” and “Is paracetamol safe for children?” received informative, safe responses. Queries asking for direct medical treatment, such as “What medication should I use for my child's fever?”, are intercepted by the safety filter and return a standard advisory message.

Key insights:

-> Prompt design shapes the model’s responses for clarity and friendliness.

-> Safety filtering is crucial for responsible deployment in health-related applications.

-> The chatbot can reliably handle general health questions without providing medical prescriptions.